# Ising fitter for capped homopolymer repeat proteins.

Authors:  Doug Barrick, Jacob D. Marold, Kathryn Geiger-Schuller, Tural Aksel, Ekaterina Poliakova-Georgantas, Sean Klein, Kevin Sforza, Mark Peterson

This notebook performs an Ising model fit to consensus Ankyrin repeat proteins (cANK). It reads data from Aviv data files, converts the data to normalized unfolding transitions, generates partition functions and expressions for fraction folded, and uses these expressions to fit the normalized transitions.  Data and fits are plotted in various ways, and bootstrap analysis is performed.  Correlation plots are generated for pairs of bootstrap parameter values.

## Imports, path, and project name

Path and project name should be set by the user.  Note that because of the kernel restart below, these must be specified in subsequent scripts, along with any other imports that are needed.

In [ ]:
from __future__ import division
import numpy as np  
import glob     
import csv
import json
import os
import time
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import lmfit
import math

path = os.getcwd() # change this to package path once we make setup.py
proj_name = 'cANK'

## Data conversion.

Data are read from an Aviv.dat file.

Outputs are

1.  A numpy data file for each melt, contining [denaturant], normalized signal, construct ID, and melt ID.

2.  A list of constructs.

3.  A list of melts.

In [ ]:
def extract_cd_signal_from_aviv(filepath):
    """Extracts the X,Y values (denaturant, CD signal) from an Aviv .dat file 
    Accepts:
        -filepath: str the filepath location of the .dat file
    Returns:
        - np.array(float[])
    """
    xylist = []
    with open(filename, 'r') as f: 
        
        lines = f.read().splitlines() #define the beginning and end of the data
        begin = 0
        end = 0
        
        while not lines[begin] == '$DATA':
            begin = begin + 1
        begin = begin + 4
        
        while not lines[end] == '$ENDDATA':
            end = end + 1    
            
        for row in range(begin, end - 1):  #extract the [denat] and CD signal
            line = lines[row]
            n = line.split()
            xylist.append([float(n[0]), float(n[1])])
    
    return np.array(xylist)

def normalize_y_values(xy):
    """Normalizes the y values of the signal
    
    Accepts: np.array(float[])
    Returns: float[]
    """
    maxval = max(xy[:,1])
    minval = min(xy[:,1])
    normylist = [float(((xy[i,1] - maxval)/(minval - maxval))) for i in range(len(xy))]
    return normylist


def organize_constructs_by_name(melts):
    ''' 
    This loop puts melts in order of type (NRxC, NRx, RxC) and length.  This is useful for the
    plotting script below, putting the by_melt legends in a sensible order
    
    Accepts: str[] - construct names
    Returns: str[] - ordered construct names
    '''
    NRClist = []
    NRlist = []
    RClist = []
    melts.sort()  # Puts in order based on length
    for melt in melts:
        if melt[0] == 'N':
            if melt[-3] == 'C':
                NRClist.append(melt)
            else:
                NRlist.append(melt)
        else:
            RClist.append(melt)
            
    melts = NRClist + NRlist + RClist
    return melts

In [ ]:
start = time.time()

den_nsig_const_melt = []
constructs = [] # List of constructs used to build partition functions and
melts = []  # List of melts to be used in fitting.
den_nsig_const_melt_df = []

# Gets file names, and extracts information including construct name, melt number.      
for num, filename in enumerate(glob.glob(os.path.join(path, "NRC_data", "*.dat"))):
    num = num + 1
    base = os.path.basename(filename)
    melt = base.split(".")[0]
    
    # Store the names of each construct to map to partition functions
    construct_name = melt[:-2]
    if construct_name not in constructs: 
        constructs.append(construct_name)

    # Reads the data portion of Aviv file
    xyarray = extract_cd_signal_from_aviv(filename)

    # Normalize the y-values from 0-1                        
    normylist = normalize_y_values(xyarray)
    
    # a melt number to use as an ID for fitting in ising script.
    single_melt_dncm = []
    for i in range(len(xyarray)): 
        x_y_name_num = [xyarray[i,0], normylist[i], construct_name, num]
        den_nsig_const_melt.append(x_y_name_num)
        single_melt_dncm.append(x_y_name_num)

    # Build a numpy array for each melt and output for Ising fitter.  
    # Columns are denaturant, normalized CD, construct, melt number.
    melt_array = np.array(single_melt_dncm)
    np.save(os.path.join(path, melt), melt_array) # Writes an npy file to disk for each melt.
    melt_df = pd.DataFrame(melt_array, columns=['denat','signal','construct_melt','dataset'])
    den_nsig_const_melt_df.append(melt_df)
    melts.append(melt)        

den_nsig_const_melt_df = pd.concat(den_nsig_const_melt_df)
den_nsig_const_melt_df.to_csv(os.path.join(path, f"{proj_name}_combined_data.csv"), index=False, header=False)

melts = organize_constructs_by_name(melts)

# Write out the results.  
with open(os.path.join(path, f"{proj_name}_constructs.json"), 'w') as r:
    json.dump(constructs, r)

with open(os.path.join(path, f"{proj_name}_melts.json"), 'w') as s:
    json.dump(melts, s)  

stop = time.time()
runtime = stop - start
print('\nThe elapsed time was ' + str(runtime) + ' sec')        

In [ ]:
den_nsig_const_melt_df

## Generate a partition function and fraction folded expressions for fitting.

Inputs the constructs.json, melts.json, and processed .npy data files from the data processing script above.

Generates a dictionary of partition functions using the capped homopolymer 1D-Ising model, and converts these to dictionaries of fraction-folded expressions (**fraction_folded_dict**) for fitting by partial differentiation.  Manipulations are done using the sympy module which allows symbolic math operations.  This is important for partial differentiation, but also for "simplification" of the fraction folded exprssions.  This simplification factors common terms, significantly decreasing the time it takes to fit and bootstrap below.  The fraction-folded dictionary is exported in json format.

Because the numpy exponential function (np.exp) gets reassigned in this script, and I cannot figure out how to undo this, the kernel must be restarted at the bottom of the script (exit()).  The user will be prompted to accept.

Though the path, project name, and most (but not all imports) are redundant with the command above, the kernel restart at the end of this script can create problems, if the script is run more than once.  For this reason I am keeping them associated with this script (and with subsequent scripts--fitting, plotting, etc).

Note that on 2020_05_05, I am changing the equation that gives the denaturant dependence of DGi to DGi + mi denat in the three equations for N, R, and C.  This corresponds to a positive m-value (free energies become more positive with denaturant).  Also change initial guess in the fitting cell.

In [ ]:
proj_name = 'cANK'

start = time.time()

print('Generating partition functions and fraction folded expressions.  This may take a minute...')

# Parameters for partition function calculation.  Note these are sympy symbols.
RT = sp.Symbol('RT')
dGN = sp.Symbol('dGN')
dGR = sp.Symbol('dGR')
dGC = sp.Symbol('dGC')
mi = sp.Symbol('mi')
denat = sp.Symbol('denat')
Kn = sp.Symbol('Kn')
Kr = sp.Symbol('Kr')
Kc = sp.Symbol('Kc')
dGinter = sp.Symbol('dGinter')
W = sp.Symbol('W')

#np.exp = sp.Function('np.exp')
exp = sp.Function('np.exp')

with open(os.path.join(path, f'{proj_name}_constructs.json'), 'r') as cons:
    constructs = json.load(cons)

#define matricies  and end vectors to be used to calculate partition functions
begin = sp.Matrix([[0,1]])
N = sp.Matrix([[(Kn*W),1],[Kn,1]])
R = sp.Matrix([[(Kr*W),1],[Kr,1]])
C = sp.Matrix([[(Kc*W),1],[Kc,1]])
end = sp.Matrix([[1],[1]])

# Build dictionaries of partition functions, partial derivs with respect
# to K, and fraction folded.

q_dict = {}
dqdKn_dict = {}
dqdKr_dict = {}
dqdKc_dict = {}
frac_folded_dict = {}

# Number of repeats of each type.  Seems like they should be floats, but
# I get an error in the matrix multiplication (q_dict) if they are declared to be.

for construct in constructs:

    # Make partition function dictionary and expressions for fraction folded. 
    # Note, only one pf is generated per construct, even when there are multiple melts.
    
    matrixlist = construct.split('_')
    q_dict[construct + '_q'] = begin 
        
    for i in range(0,len(matrixlist)):
        num_Ni = 0
        num_Ri = 0
        num_Ci = 0
        if matrixlist[i] == 'N':
            num_Ni=1
        if matrixlist[i] == 'R':
            num_Ri=1
        if matrixlist[i] == 'C':
            num_Ci=1

        q_dict[construct + '_q'] = q_dict[construct + '_q'] *\
        np.linalg.matrix_power(N, num_Ni) * np.linalg.matrix_power(R, num_Ri) *\
        np.linalg.matrix_power(C, num_Ci) 
      
    q_dict[construct + '_q'] =  q_dict[construct + '_q'] * end

    # Next two lines convert from sp.Matrix to np.array to something else.
    # Not sure the logic here, but it works.
    
    q_dict[construct + '_q'] = np.array(q_dict[construct + '_q']) 
    q_dict[construct + '_q'] = q_dict[construct + '_q'].item(0)

    # Partial derivs wrt Kn dictionary.
    dqdKn_dict[construct + '_dqdKn'] \
        = sp.diff(q_dict[construct + '_q'], Kn)

    # Partial derivs wrt Kr dictionary.
    dqdKr_dict[construct + '_dqdKr'] \
        = sp.diff(q_dict[construct + '_q'], Kr)

    # Partial derivs wrt Kc dictionary.
    dqdKc_dict[construct + '_dqdKc'] \
        = sp.diff(q_dict[construct + '_q'], Kc)

    # Fraction folded dictionary.  
    frac_folded_dict[construct + '_frac_folded'] \
        = (Kn/( q_dict[construct + '_q']) * dqdKn_dict[construct + '_dqdKn'] \
        + Kr/(q_dict[construct + '_q']) * dqdKr_dict[construct + '_dqdKr'] \
        + Kc/( q_dict[construct + '_q']) * dqdKc_dict[construct + '_dqdKc']) \
        / (len(matrixlist))

# The loop below replaces K's and W's the fraction folded terms in the 
# dictionary with DGs, ms, and denaturant concentrations.  The simplify line
# is really important for making compact expressions for fraction folded.
# This simplification greatly speeds up fitting.  The last line
# converts from a sympy object to a string, to allow for json dump.

for construct in frac_folded_dict:
    frac_folded_dict[construct] = frac_folded_dict[construct].subs({
    Kn:(exp(-((dGN + (mi*denat))/RT))), 
    Kr:(exp(-((dGR + (mi*denat))/RT))), 
    Kc:(exp(-((dGC + (mi*denat))/RT))),  
    W:(exp(-dGinter/RT)) }) 
    frac_folded_dict[construct] = sp.simplify(frac_folded_dict[construct])
    frac_folded_dict[construct] = str(frac_folded_dict[construct])                                

with open(os.path.join(path, f'{proj_name}_frac_folded_dict.json'), 'w') as f:
    json.dump(frac_folded_dict, f)
   
stop = time.time()
runtime = stop - start
print('\nThe elapsed time was ' + str(runtime) + ' sec') 

## Fitting the data with the Ising model

Processed data files are imported along with the fraction-folded dictionary and construct and melt lists.  The fit is performed with the lmfit module, which has extra functionality over fitting routines in scipy.  

Note that if your initial guesses are poor, the fit may be slowed significantly or the fit may not converge.

Fitted thermodynamic parameters are outputted to the screen and are written to a csv file.  Baseline parameters are also written to a csv file.

In [ ]:
print("\nFitting the data...\n")

start = time.time()

plt.close()
plt.clf

RT = 0.001987 * 298.15 #  R in kcal/mol/K, T in Kelvin.

#  Dictionary of frac folded eqns from partition function generator script.
with open(os.path.join(path, f'{proj_name}_frac_folded_dict.json'), 'r') as ffd:
    frac_folded_dict = json.load(ffd)

with open(os.path.join(path, f'{proj_name}_constructs.json'), 'r') as construct:
    constructs = json.load(construct)

with open(os.path.join(path, f'{proj_name}_melts.json'), 'r') as m:
    melts = json.load(m)

num_melts = len(melts)
num_constructs = len(constructs)
melt_data_dict = {melt: np.load(os.path.join(path, f'{melt}.npy')) for melt in melts}


# Compile fraction folded expressions.
comp_frac_folded_dict = {}
for construct in constructs:
    frac_folded_string = frac_folded_dict[construct + '_frac_folded']
    comp_frac_folded = compile(frac_folded_string, '{}_comp_ff'.format(construct), 'eval')
    comp_frac_folded_dict[construct + '_comp_ff'] = comp_frac_folded #comp_frac_folded

# CREATE INITIAL GUESSES
# First, thermodynamic parameters.  These are Global.
init_guesses = lmfit.Parameters()
init_guesses.add('dGN', value = 6)
init_guesses.add('dGR', value = 5)
init_guesses.add('dGC', value = 6)
init_guesses.add('dGinter', value = -12)
init_guesses.add('mi', value = 1.0)

# Next, baseline parameters.  These are local.
for melt in melts:
    init_guesses.add('af_{}'.format(melt), value=0.02)
    init_guesses.add('bf_{}'.format(melt), value=1)
    init_guesses.add('au_{}'.format(melt), value=0.0)
    init_guesses.add('bu_{}'.format(melt), value=0.0)

# Transfers init_guesses to params for fitting, but init_guesses are maintained.
params = init_guesses

def fitting_function(params, denat, frac_folded, melt):
    af = params['af_{}'.format(melt)].value
    bf = params['bf_{}'.format(melt)].value
    au = params['au_{}'.format(melt)].value
    bu = params['bu_{}'.format(melt)].value
    dGN = params['dGN'].value
    dGR = params['dGR'].value
    dGC = params['dGC'].value
    dGinter = params['dGinter'].value
    mi = params['mi'].value
    return ((af * denat) + bf) * frac_folded + (((au * denat) + bu) * (1 - frac_folded))

# Objective function creates an array of residuals to be used by lmfit minimize.
def objective(params):
    resid_dict = {}
    dGN = params['dGN'].value  
    dGR = params['dGR'].value   
    dGC = params['dGC'].value
    dGinter = params['dGinter'].value
    mi = params['mi'].value
    for melt in melts:   
        denat = melt_data_dict[melt][:,0]     # A numpy array of type str
        norm_sig = melt_data_dict[melt][:,1]  # A numpy array of type str
        denat = denat.astype(float) # A numpy array of type float
        norm_sig = norm_sig.astype(float) # A numpy array of type float
       
        string_to_eval = comp_frac_folded_dict[melt[:-2] + '_comp_ff']
        frac_folded = eval(string_to_eval)
        
        # frac_folded name gets associated for use in fitting_function call in frac_folded_string assignment above.
        af = params['af_{}'.format(melt)].value
        bf = params['bf_{}'.format(melt)].value
        au = params['au_{}'.format(melt)].value
        bu = params['bu_{}'.format(melt)].value
        resid = norm_sig - fitting_function(params, denat, frac_folded, melt)
        resid_dict[melt + '_resid'] = resid
    residuals = np.concatenate(list(resid_dict.values()))
    return residuals

# Fit with lmfit
result = lmfit.minimize(objective, init_guesses)
fit_resid = result.residual

# Print out features of the data, the fit, and optimized param values
print("There are a total of {} data sets.".format(num_melts))
print("There are {} observations.".format(result.ndata))
print("There are {} fitted parameters.".format(result.nvarys))
print("There are {} degrees of freedom. \n".format(result.nfree))
print("The sum of squared residuals (SSR) is: {0:7.4f}".format(result.chisqr))
print("The reduced SSR (SSR/DOF): {0:8.6f} \n".format(result.redchi))

dGN = result.params['dGN'].value
dGR = result.params['dGR'].value
dGC = result.params['dGC'].value
dGinter = result.params['dGinter'].value
mi = result.params['mi'].value

print('Optimized parameter values:')
print('dGN = {0:8.4f}'.format(result.params['dGN'].value))
print('dGR = {0:8.4f}'.format(result.params['dGR'].value))
print('dGC ={0:8.4f}'.format(result.params['dGC'].value))
print('dGinter ={0:8.4f}'.format(result.params['dGinter'].value))
print('mi ={0:8.4f}'.format(result.params['mi'].value))

print("\nWriting best fit parameter and baseline files")

# Compile a list of optimized Ising params and write to file.
fitted_ising_params = [["dGN", result.params['dGN'].value], 
                        ["dGR", result.params['dGR'].value], 
                        ["dGC", result.params['dGC'].value], 
                        ["dGinter", result.params['dGinter'].value],
                        ["mi", result.params['mi'].value], 
                        ["Chi**2",result.chisqr],
                        ["RedChi",result.redchi]]

with open(os.path.join(path, f'{proj_name}_fitted_Ising_params.csv'), "w") as n:
    writer = csv.writer(n, delimiter=',')
    writer.writerows(fitted_ising_params)
n.close()

# Compile a list of optimized baseline params and write to file.
fitted_base_params = []
for melt in melts:
    af = result.params['af_%s' % (melt)].value            
    bf = result.params['bf_%s' % (melt)].value              
    au = result.params['au_%s' % (melt)].value          
    bu = result.params['bu_%s' % (melt)].value           
    fitted_base_params.append([melt, af, bf, au, bu])            
with open(os.path.join(path, f'{proj_name}_fitted_baseline_params.csv'), "w") as m:              
    writer = csv.writer(m, delimiter=',')
    writer.writerows(fitted_base_params) 
m.close()

stop = time.time()
runtime = stop - start
print('\nThe elapsed time was ' + str(runtime) + ' sec')

## Plotting the results of the fit

This cell generates four plots.  Two are "normalized" data (the data that were actually fit in the scipt above) and fits.  The other two are fraction-folded data and fits.  One each shows all the constructs, which ideally includes multiple melts of each construct, allowing all fits to be inspected.  The other shows only a single melt for each construct (the first one in the melt list for each), simplifying the plot.

The resulting plots are dumped to the screen below the cell, and are saved as png files.

Note that this script is meant to be run after the fitting script.  If the fit has not been performed in the current session (or the kernel was restarted after the fit--*not usually the case*), then imports will have to be run, along with data and fitted parameters.  That would be pain, so just re-run the fit again, if you find yourself in this situation.

In [ ]:
print("\nPlotting results...\n")

# The function "baseline_adj" gives an adjusted y value based on fitted baseline 
# parameters (fraction folded). 
def baseline_adj(y, x, params, construct):
    af = result.params['af_{}'.format(construct)].value
    bf = result.params['bf_{}'.format(construct)].value
    au = result.params['au_{}'.format(construct)].value
    bu = result.params['bu_{}'.format(construct)].value
    return (y-(bu+(au*x)))/((bf+(af*x))-(bu+(au*x)))

# Defining global best-fit parameters
dGN = result.params['dGN'].value
dGR = result.params['dGR'].value
dGC = result.params['dGC'].value
dGinter =result.params['dGinter'].value
mi = result.params['mi'].value

# The function fit_model used for plotting best-fit lines and for adding  
# residuals to best-fit lines in bootstrapping.  Normalized, not frac folded.
def fit_model(params, x, melt):
    denat = x
    af = result.params['af_{}'.format(melt)].value
    bf = result.params['bf_{}'.format(melt)].value
    au = result.params['au_{}'.format(melt)].value
    bu = result.params['bu_{}'.format(melt)].value
    frac_folded = eval(comp_frac_folded_dict[melt[:-2] + '_comp_ff']) # :-2 leaves off the _1, _2, etc from melt id.    
    return ((af * denat) + bf) * frac_folded + (((au * denat) + bu) * \
            (1 - frac_folded))   

# Finding the maximum denaturant value out of all the melts to
# set x axis bound
denat_maxer = np.zeros(0)
for melt in melts:
    denat_maxer = np.concatenate((denat_maxer, melt_data_dict[melt][:, 0]))
denat_maxer_list = denat_maxer.tolist()
denat_max = float(max(denat_maxer_list))
denat_bound = np.around(denat_max, 1) + 0.2

# Denaturant values to use when evaluating fits.  Determines how smooth the
# fitted curve will be, based on the third value (300) in the argument below.
# I might keep using this for fraction_foldeed, but for nomralized baseline
# use a local set of points for each melt, so as not to extrapolate the 
# bselines too far.
denat_fit = np.linspace(0, denat_bound, 300)

#defining a dictionary using the first melt of each construct (construct_1)
#Move this to the plotting part, and why not do this for all constructs?
construct1_data_dict = {}
for construct in constructs:
    construct1_data_dict[construct] = np.load(os.path.join(path, f'{construct}_1.npy'))

# The four dictionaries below define lower and upper denaturant limnits to be
# used for plotting normalized curves, so crazy-long baseline extrapolations 
# are not shown.  Do both for melts and construct 1.   These are then used
# to create 300-point synthetic baselines in the fifth and sixth dictionaries.
melt_lower_denat_dict = {}
for melt in melts:
    melt_lower_denat_dict[melt] = round(float(min(melt_data_dict[melt][:,0]))) -0.2

melt_upper_denat_dict = {}
for melt in melts:
    melt_upper_denat_dict[melt] = round(float(max(melt_data_dict[melt][:,0]))) + 0.2

construct1_lower_denat_dict = {}
for construct in constructs:
    construct1_lower_denat_dict[construct] = round(float(min(construct1_data_dict[construct][:,0]))) - 0.2

construct1_upper_denat_dict = {}
for construct in constructs:
    construct1_upper_denat_dict[construct] = round(float(max(construct1_data_dict[construct][:,0]))) + 0.2

melt_denat_synthetic_dict = {}
for melt in melts:
    melt_denat_synthetic_dict[melt] = np.linspace(melt_lower_denat_dict[melt],
                             melt_upper_denat_dict[melt], 300)

construct1_denat_synthetic_dict = {}
for construct in constructs:
    construct1_denat_synthetic_dict[construct] = np.linspace(construct1_lower_denat_dict[construct],
                                                             construct1_upper_denat_dict[construct], 300)

''' Global Plot Aesthetics'''
# Defining how the plots are colored
num_melt_colors = num_melts
num_construct_colors = num_constructs
coloration = plt.get_cmap('hsv')


# Dictonary defining title font
title_font = {
    'family': 'arial',
    'color': 'black',
    'weight': 'normal',
    'size': 16
}

# Dictionary defining label font
label_font = {
    'family': 'arial',
    'color': 'black',
    'weight': 'normal',
    'size': 14
}

'''First Plot: Fraction Folded by Melt'''
#extracting the melt data and creating plot lines for each melt
colorset = 0 # counter to control color of curves and points
for melt in melts:   
    colorset = colorset + 1
    denat = melt_data_dict[melt][:,0]     # A numpy array of type str
    norm_sig = melt_data_dict[melt][:,1]  # A numpy array of type str
    denat = denat.astype(float) # A numpy array of type float
    norm_sig = norm_sig.astype(float) # A numpy array of type float
    y_adj = baseline_adj(norm_sig, denat, result.params, melt)
    y_fit = fit_model(result.params, denat_fit, melt)
    y_fit_adj = baseline_adj(y_fit, denat_fit, result.params, melt)
    plt.plot(denat, y_adj, 'o', color = coloration(colorset/num_melt_colors),
            label = melt[:-2] + ' melt ' + melt[-1])       
    plt.plot(denat_fit, y_fit_adj, '-', color = coloration(colorset/num_melt_colors))

#set axis limits
axes=plt.gca()
axes.set_xlim([-0.1, denat_bound])
axes.set_ylim([-0.1,1.1])
axes.set_aspect(5.5)

#lot aesthetics and labels
plt.legend(loc = 'center', bbox_to_anchor = (1.25, 0.5), fontsize=8)
plt.title('Fraction Folded by Melt', fontdict = title_font)
plt.xlabel('Denaturant (Molar)', fontdict = label_font)
plt.ylabel('Fraction Folded', fontdict = label_font)

#saving plot in individual doc
plt.savefig(os.path.join(path, f'{proj_name}_plot_frac_folded_by_melt.png'),\
            dpi = 500, bbox_inches='tight')

#show plot in iPython window and then close
plt.show()
plt.close()
plt.clf

'''Second Plot: Normalized Signal by Melt'''
colorset = 0
for melt in melts:   
    colorset = colorset + 1
    denat = melt_data_dict[melt][:,0]     # A numpy array of type str
    norm_sig = melt_data_dict[melt][:,1]  # A numpy array of type str
    denat = denat.astype(float) # A numpy array of type float
    norm_sig = norm_sig.astype(float) # A numpy array of type float
    y_fit = fit_model(result.params, melt_denat_synthetic_dict[melt], melt)
    plt.plot(denat, norm_sig, 'o', color=coloration(colorset/num_melt_colors),
             label = melt[:-2] + ' melt ' + melt[-1]) 
    plt.plot(melt_denat_synthetic_dict[melt], y_fit, '-', \
             color=coloration(colorset/num_melt_colors))

#set axis limits
axes=plt.gca()
axes.set_xlim([-0.1, denat_bound])
axes.set_ylim([-0.1,1.1])
axes.set_aspect(5.5)

#plot aesthetics and labels
plt.legend(loc = 'center', bbox_to_anchor = (1.25, 0.5), fontsize=8)
plt.title('Normalized Signal by Melt', fontdict = title_font)
plt.xlabel('Denaturant (Molar)', fontdict = label_font)
plt.ylabel('Normalized Signal', fontdict = label_font)

#saving plot in individual doc
plt.savefig(os.path.join(path, f'{proj_name}_plot_normalized_by_melt.png'),\
            dpi=500, bbox_inches='tight')

#show plot in iPython window and then close
plt.show()
plt.close()
plt.clf

'''Third Plot: Fraction Folded by Construct'''
colorset = 0
for construct in constructs:   
    colorset = colorset + 1
    denat = construct1_data_dict[construct][:,0]  # A numpy array of type str
    denat_line = construct1_data_dict[construct][:, 0]   # A numpy array of type str
    norm_sig = construct1_data_dict[construct][:, 1]  # A numpy array of type str
    denat = denat.astype(float) # A numpy array of type float
    denat_line = denat_line.astype(float) # A numpy array of type float
    norm_sig = norm_sig.astype(float) # A numpy array of type float
    y_adj = baseline_adj(norm_sig, denat_line, result.params, construct + '_1')
    y_fit = fit_model(result.params, denat_fit, construct + '_1')
    y_fit_adj = baseline_adj(y_fit, denat_fit, result.params, construct + '_1')
    plt.plot(denat, y_adj, 'o', \
             color=coloration(colorset/num_construct_colors), label = construct)       
    plt.plot(denat_fit, y_fit_adj, '-', \
             color=coloration(colorset/num_construct_colors))

#set axis limits
axes=plt.gca()
axes.set_xlim([-0.1, denat_bound])
axes.set_ylim([-0.1,1.1])
axes.set_aspect(5.5)

#plot aesthetics and labels
plt.legend(loc = 'center', bbox_to_anchor = (1.15, 0.5), fontsize=8)
plt.title('Fraction Folded by Construct', fontdict = title_font)
plt.xlabel('Denaturant (Molar)', fontdict = label_font)
plt.ylabel('Fraction Folded', fontdict = label_font)

#saving plot in individual doc
plt.savefig(os.path.join(path, f'{proj_name}_plot_frac_folded_by_construct.png'),\
            dpi=500, bbox_inches='tight')

#show plot in iPython window and then close
plt.show()
plt.close()
plt.clf

'''Fourth Plot: Normalized Signal by Construct'''
colorset = 0
for construct in constructs:   
    colorset = colorset + 1
    denat = construct1_data_dict[construct][:,0]     # A numpy array of type str
    norm_sig = construct1_data_dict[construct][:,1]  # A numpy array of type str
    denat = denat.astype(float) # A numpy array of type float
    norm_sig = norm_sig.astype(float) # A numpy array of type float
    y_fit = fit_model(result.params, construct1_denat_synthetic_dict[construct], \
                      construct + '_1')
    plt.plot(denat, norm_sig, 'o', color = coloration(colorset/num_construct_colors),
             label = construct) 
    plt.plot(construct1_denat_synthetic_dict[construct], y_fit, '-', \
             color = coloration(colorset/num_construct_colors))

#set axis limits
axes=plt.gca()
axes.set_xlim([-0.1, denat_bound])
axes.set_ylim([-0.1,1.1])
axes.set_aspect(5.5)

#plot aesthetics and labels
plt.legend(loc = 'center', bbox_to_anchor = (1.15, 0.5), fontsize=8)
plt.title('Normalized Signal by Construct', fontdict = title_font)
plt.xlabel('Denaturant (Molar)', fontdict = label_font)
plt.ylabel('Normalized Signal', fontdict = label_font)

#saving plot in individual doc
plt.savefig(os.path.join(path, f'{proj_name}_plot_normalized_by_construct.png'),\
            dpi=500, bbox_inches='tight')

#show plot in iPython window and then close
plt.show()
plt.close()
plt.clf

## Bootstrap analysis

Asks the user to input the number of bootstrap iterations.  Bootstrap parameters are stored in a list of lists (**bs_param_values**). After performing the specified number of iterations, bootstrapped thermodynamic parameters are written to a csv file.

Again, bootstrapping is meant to be performed after fitting above.  Otherwise, the data and the fit model will have to be re-imported, and the params list and objective function will need to be generated.  Just run the fit again if needed.

In this version, a two minute sleep command is built in every 50 bootstrap iterations to let things cool down.

In [ ]:
'''BootStrap analysis'''
# Create list to store bootstrap iterations of values and define column titles
bs_param_values = []  
bs_param_values.append(['Bootstrap Iter', 'dGN', 'dGR', 'dGC', 'dGinter', 'mi', 
                        'redchi**2','bestchi**2'])   

#total number of bootstrap iterations 
bs_iter_tot = input("How many bootstrap iterations? ") 

# bs_iter_tot = 10  # You would use this if you did not want user input from screen
bs_iter_count = 0   # Iteration counter
fit_resid_index= len(fit_resid) - 1

y_fitted_dict = {}
# Dictionary of 'true' normalized y values from fit at each denaturant value. 
for melt in melts:
    denat = melt_data_dict[melt][:,0] # A numpy array of type str
    denat = denat.astype(float)       # A numpy array of type float
    y_fitted_dict[melt] = np.array(fit_model(result.params, denat, melt))

# Arrays to store bs fitted param values
dGN_vals = []
dGR_vals = []
dGC_vals = []
dGinter_vals = []
mi_vals = []

# Add residuals chosen at random (with replacement) to expected
# y values. Note-residuals are combined ACROSS melts.
for j in range(int(bs_iter_tot)):   
    rand_resid_dict={}  # Clears the random data for each bootsterap iteration
    bs_iter_count = bs_iter_count + 1
    print("Bootstrap iteration {0} out of {1}".format(bs_iter_count, 
                               bs_iter_tot))
    
    for melt in melts:
        rand_resid =[]
        denat = melt_data_dict[melt][:,0] # A numpy array of type str
        denat = denat.astype(float)       # A numpy array of type float
        
        for x in range(0,len(denat)):  # Creastes a list of random residuals
            rand_int = np.random.randint(0, fit_resid_index)
            rand_resid.append(fit_resid[rand_int])  
 
        rand_resid_dict[melt] = np.array(rand_resid)
        y_bootstrap = y_fitted_dict[melt] + rand_resid_dict[melt]
        z_max,z_min = y_bootstrap.max(), y_bootstrap.min()
        melt_data_dict[melt][:, 1] = (y_bootstrap - z_min)/(z_max - z_min)
    
    bs_result = lmfit.minimize(objective, init_guesses)
    bs_chisqr = bs_result.chisqr
    bs_red_chisqr= bs_result.redchi
    
    dGN = bs_result.params['dGN'].value
    dGR = bs_result.params['dGR'].value
    dGC = bs_result.params['dGC'].value
    dGinter = bs_result.params['dGinter'].value
    mi = bs_result.params['mi'].value
    
    # Store each value in a list for plotting and for downstream statistical analysis
    dGN_vals.append(dGN)
    dGR_vals.append(dGR)
    dGC_vals.append(dGC)
    dGinter_vals.append(dGinter)
    mi_vals.append(mi)
    
    # Append bootstrapped global parameter values for ouput to a file
    bs_param_values.append([bs_iter_count, dGN, dGR, dGC, dGinter, mi, 
                            bs_red_chisqr,bs_chisqr])
        
    
with open(os.path.join(path, f'{proj_name}_bootstrap_params.csv'), "w") as n:
    writer = csv.writer(n, delimiter = ',')
    writer.writerows(bs_param_values)
n.close()

## The next cell calculates statistical properties of bootstrap parameters and outputs a file

I plan to merge this with the bootstrap cell, but it is much more convenient to code it separately.  

The structure that currently holds the bootstrap parameter values (*bs_param_values*) is a list of lists.  So it needs to be converted to a numpy array, and it needs to have only values, not column heads, in order to do numerical calculations.  Pandas would clearly be the right way to go with this, but not today.

*path* (for writing out the data frame) is taken from the fitting cell above.

In [ ]:
bs_param_values_fullarray = np.array(bs_param_values)
bs_param_values_array = bs_param_values_fullarray[1:,1:-2].astype(np.float) # End at -2 since last two columns
                                                                            # are chi square statistics

bs_param_names = bs_param_values_fullarray[0][1:-2]

statistics = ['mean','median','stdev','2.5% CI','16.6% CI','83.7% CI','97.5% CI']

bs_statistics_df = pd.DataFrame(columns = statistics)

i = 0
for param in bs_param_names:
    bs_statistics = []
    bs_statistics.append(np.mean(bs_param_values_array[:,i]))
    bs_statistics.append(np.median(bs_param_values_array[:,i]))
    bs_statistics.append(np.std(bs_param_values_array[:,i]))
    bs_statistics.append(np.percentile(bs_param_values_array[:,i],2.5))
    bs_statistics.append(np.percentile(bs_param_values_array[:,i],16.7))
    bs_statistics.append(np.percentile(bs_param_values_array[:,i],83.3))
    bs_statistics.append(np.percentile(bs_param_values_array[:,i],97.5))
    bs_statistics_df.loc[param] = bs_statistics
    i = i + 1

bs_statistics_df.to_csv(os.path.join(path, f'{proj_name}_bootstrap_stats.csv'))

corr_coef_matrix = np.corrcoef(bs_param_values_array, rowvar = False)
corr_coef_df = pd.DataFrame(corr_coef_matrix, columns = bs_param_names, index = bs_param_names)
corr_coef_df.to_csv(os.path.join(path, f'{proj_name}_bootstrap_corr_coefs.csv'))

In [ ]:
bs_statistics_df

In [ ]:
corr_coef_matrix

In [ ]:
corr_coef_df

## Bootstrap histograms and correlation plots

Plots are generated for the thermodynamic parameters of interest (currently, baseline parameters are not included, thought this would not be hard to generate).  Histograms are generated for each parameter.  Scatter plots are generated for each pair of parameters (not including self-correlation) and arrayed in a grid along with a linear fit.  Shared axes are used in the grid to minimize white-space resulting from labelling each axis.  Thinking about the output as a matrix, the histograms are on the main diagonal, and the correllation plots are off-diagonal elements populating the upper triangle of the matrix.

The plot grid is dumped to the screen below, and is also written as a pdf file.

As with the plotting and bootstrapping scripts above, this is meant to be run after the fitting script (and after the bootstrapping script immediately above).  If you have not done that, re-run fit and bootstrap.

In [ ]:
# Specify the names of parameters to be compared to see correlation.
corr_params = ['dGN', 'dGR', 'dGC', 'dGinter', 'mi']

# These are a second set of parameter names that follow in the same order
# as in corr_params.  They are formatted using TeX-style names so that Deltas 
# and subscripts will be plotted.  The would not be good key names for dictionaries
corr_param_labels = ['$\Delta$G$_N$', '$\Delta$G$_R$', '$\Delta$G$_C$', 
               '$\Delta$G$_{i, i-1}$', 'm$_i$']

num_corr_params = len(corr_params)
gridsize = num_corr_params  # Determines the size of the plot grid.

# Dictionary of fitted parameter values.
corr_params_dict = {'dGN': dGN_vals, 'dGR': dGR_vals, 'dGC': dGC_vals,\
                    'dGinter': dGinter_vals, 'mi': mi_vals}

# PDF that stores a grid of the correlation plots
with PdfPages(os.path.join(path, f'{proj_name}_Corr_Plots.pdf')) as pdf:
    fig, axs = plt.subplots(ncols=gridsize, nrows=gridsize, figsize=(12, 12))
    
    # Turns off axes on lower triangle
    axs[1, 0].axis('off')
    axs[2, 0].axis('off')
    axs[2, 1].axis('off')
    axs[3, 0].axis('off')
    axs[3, 1].axis('off')
    axs[3, 2].axis('off')
    axs[4, 0].axis('off')
    axs[4, 1].axis('off')
    axs[4, 2].axis('off')
    axs[4, 3].axis('off')

    # Defines the position of the y paramater from the array of params
    hist_param_counter = 0
    while hist_param_counter < num_corr_params:
        hist_param_label = corr_param_labels[hist_param_counter]
        hist_param = corr_params[hist_param_counter]
        # Start fixing labels here
        #plt.xticks(fontsize=8)
        #axs[hist_param_counter, hist_param_counter].tick_params(fontsize=8)
        #axs[hist_param_counter, hist_param_counter].yticks(fontsize=8)
        axs[hist_param_counter, hist_param_counter].hist(corr_params_dict[hist_param])
        axs[hist_param_counter, hist_param_counter].set_xlabel(hist_param_label,
                   fontsize=14, labelpad = 5)
        hist_param_counter = hist_param_counter + 1  
    
    # This part generates the correlation plots
    y_param_counter = 0
    while y_param_counter < num_corr_params - 1:
        # Pulls the parameter name for the y-axis label (with TeX formatting)
        yparam_label = corr_param_labels[y_param_counter]
        # Pulls the parameter name to be plotted on the y-axis
        yparam = corr_params[y_param_counter]
        
        # Defines the position of the x paramater from the array of params.
        # The + 1 offest avoids correlating a parameter with itself.
        x_param_counter = y_param_counter + 1
        
        while (x_param_counter < num_corr_params):
            #pulls the parameter name for the x-axis label (with TeX formatting)
            xparam_label = corr_param_labels[x_param_counter]
            # Pulls the parameter name to be plotted on the x-axis
            xparam = corr_params[x_param_counter]
            
            x_vals= corr_params_dict[xparam]
            y_vals = corr_params_dict[yparam]
            
            #plt.xticks(fontsize=8)
            #plt.yticks(fontsize=8)
            #plotting scatters with axes.  +1 shifts a plot to the right from main diagonal
            axs[y_param_counter, x_param_counter].plot(x_vals, y_vals, '.')
            
            # The if statement below turns off numbers on axes if not the right column and
            # not the main diagonal.
            if x_param_counter < num_corr_params - 1:
                axs[y_param_counter, x_param_counter].set_xticklabels([])
                axs[y_param_counter, x_param_counter].set_yticklabels([])
                           
            if y_param_counter == 0:  # Puts labels above axes on top row
                axs[y_param_counter, x_param_counter].xaxis.set_label_position('top')
                axs[y_param_counter, x_param_counter].set_xlabel(xparam_label,
                   labelpad = 10, fontsize=14)
                axs[y_param_counter, x_param_counter].xaxis.tick_top()
                if x_param_counter < num_corr_params - 1:  # Avoids eliminating y-scale from upper right corner
                    axs[y_param_counter, x_param_counter].set_yticklabels([])
           
            if x_param_counter == num_corr_params - 1:  #  Puts labels right of right column
                axs[y_param_counter, x_param_counter].yaxis.set_label_position('right')
                axs[y_param_counter, x_param_counter].set_ylabel(yparam_label, 
                   rotation = 0, labelpad = 30, fontsize=14)
                axs[y_param_counter, x_param_counter].set_xticklabels([])
                axs[y_param_counter, x_param_counter].yaxis.tick_right()
               
            # Determin correlation coefficient and display under subplot title
            # Note, there is no code that displays this value at the moment.
            #corr_coef = np.around(np.corrcoef(x_vals, y_vals), 3)
            
            #min and max values of the x param
            x_min = min(x_vals)
            x_max = max(x_vals)
            
            #fitting a straight line to the correlation scatterplot
            fit_array = np.polyfit(x_vals, y_vals, 1)
            fit_deg1_coef = fit_array[0]
            fit_deg0_coef = fit_array[1]      
            fit_x_vals = np.linspace(x_min, x_max, 10)
            fit_y_vals = fit_deg1_coef*fit_x_vals + fit_deg0_coef
            
            #plotting correlation line fits
            axs[y_param_counter, x_param_counter].plot(fit_x_vals, 
               fit_y_vals)
            plt.subplots_adjust(wspace=0, hspace=0)
            
            x_param_counter = x_param_counter + 1            
        y_param_counter = y_param_counter + 1
    
    pdf.savefig(bbox_inches='tight')